In [1]:
!pip install --upgrade pip
!pip install quandl


     |████████████████████████████████| 1.5 MB 739 kB/s eta 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 20.0.2
    Uninstalling pip-20.0.2:
      Successfully uninstalled pip-20.0.2


In [2]:
# import necessary libraries

import pandas as pd
import quandl



from datetime import date
from dateutil.relativedelta import relativedelta

import time

import pickle


In [3]:
quandl.ApiConfig.api_key = ""   # API key removed for github

In [4]:
# load tickers from SHARADAR/TICKERS

tic = time.time()

sharadar_tickers= quandl.get_table('SHARADAR/TICKERS', table='SF1', paginate=True )

toc=time.time()
      
print(str(toc-tic)+ ' s')

12.806714057922363 s


In [5]:
# toss out delisted tickers and apply some filters

sharadar_tickers=sharadar_tickers[sharadar_tickers['isdelisted']=='N']\
                                 [sharadar_tickers['siccode'] > 0]\
                                 [sharadar_tickers['exchange']!='OTC']
 

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [6]:
# select industries from .xls file
industries_to_remove = ['Asset Management','Banks - Diversified','Banks - Regional','Banks - Regional - US','Capital Markets', 
                       'Credit Services', 'Financial Conglomerates','Financial Data & Stock Exchanges','Financial Exchanges',
                       'Health Care Plans', 'Healthcare Plans', 'Insurance - Diversified','Insurance - Life', 
                        'Insurance - Property & Casualty', 'Insurance - Reinsurance','Insurance - Specialty','Insurance Brokers'
                       'Mortgage Finance','REIT - Diversified','REIT - Healthcare Facilities','REIT - Hotel & Motel', 
                        'REIT - Industrial', 'REIT - Mortgage','REIT - Office','REIT - Residential','REIT - Retail',
                       'REIT - Specialty','Savings & Cooperative Banks','Shell Companies','Specialty Finance']

sharadar_tickers_f=sharadar_tickers[~sharadar_tickers.industry.isin(industries_to_remove)]

In [7]:
tickers_list=sharadar_tickers_f.ticker.tolist()

In [8]:
PORT_EQUITY = 500000

RISK_FACTOR = 0.00075


n=14 # days to average



# for stock price total return calculations over y days and z days
y=10
z=20


In [9]:

n_months=n//(21)+1 # months to load

today=date.today()
s_today=str(today.year)+'-'+ str(today.month) +'-'+str(today.day)

start_date = date.today() + relativedelta(months=-n_months)   #load n_months
s_start_date=str(start_date.year)+'-'+ str(start_date.month) +'-'+str(start_date.day)


In [10]:
# pull all data from SHARADAR/SEP at once

tic = time.time()

sharadar_sep_all = quandl.get_table('SHARADAR/SEP',
                                    date={'gte':s_start_date, 'lte': s_today},
                                    qopts={"columns":["ticker","high", "low", "close","volume","dividends"]},
                                    ticker=tickers_list,
                                    paginate=True)       # load last n_months of data

toc=time.time()
      
print(str(toc-tic)+ ' s')





10.964792013168335 s


In [11]:
# filter all on the basis of (PORT_EQUITY * RISK_FACTOR * last_close) / last_atr < last_mddv * 0.05

# averaging only n last values

# month true range % filter. toss if: (21 day highest high - 21 day lowest low) / last_close < 0.03




tic = time.time()

tickers_list1=[] #new list



print(str(len(tickers_list))+' tickers')

#(latest_close - n_day_close + sum of dividends since n) / n_day_close

df_total_return=pd.DataFrame(columns = ['ticker',
                                        'latest_close',
                                        'y_day_close',
                                        'z_day_close',
                                        'y_sum_dividends',
                                        'z_sum_dividends'])



                                   


i=0

for ticker in tickers_list:
    try:  # some tickers have empty dataframes or there is a problem with loading data from quandl
       
        i+=1
        sharadar_sep=sharadar_sep_all[sharadar_sep_all['ticker'] == ticker].reset_index()
    
   
        df1=sharadar_sep.copy()[0:n]
    
        high = df1['high']
        low = df1['low']
        close = df1['close']
    
   
        df1['tr0'] = abs(high - low)
        df1['tr1'] = abs(high - close.shift())
        df1['tr2'] = abs(low - close.shift())
    
    
        tr = df1[['tr0', 'tr1', 'tr2']].max(axis=1)
    
    
        volume = sharadar_sep[0:n]['volume']*sharadar_sep[0:n]['close']
        
        try:
               
            last_close = sharadar_sep['close'][0]
            
        except IndexError:
            
            last_close = float('NaN')
        
        
        
        last_atr= tr.mean()
        last_mddv=volume.median()
    
        Filter=((PORT_EQUITY * RISK_FACTOR * last_close) / last_atr < last_mddv * 0.05)
        
        n1=21
        
        Filter1 = ( max(sharadar_sep[0:n1]['high']) - min(sharadar_sep[0:n1]['low'])  ) /sharadar_sep['close'][0]< 0.03

                      
        
        if Filter and (not Filter1):
            tickers_list1.append(ticker)
            
                     
            
            try:
            
                y_day_close = sharadar_sep['close'][y]
                z_day_close = sharadar_sep['close'][z]
                
            except KeyError:
                
                y_day_close = float('NaN')
                z_day_close = float('NaN')
            
            
            y_sum_dividends = sharadar_sep['dividends'][0:y].sum()
            z_sum_dividends = sharadar_sep['dividends'][0:z].sum()

            
                        
            df_total_return=df_total_return.append({'ticker': ticker,
                                                    'latest_close':last_close,
                                                    'y_day_close':y_day_close,
                                                    'z_day_close':z_day_close,
                                                    'y_sum_dividends':y_sum_dividends,
                                                    'z_sum_dividends':z_sum_dividends,
                                                    'ATR':last_atr
                                                    },
                                                      ignore_index=True)
                  

        if i%100==0:
            print(i)
            
            
    except ValueError:
        pass

print(str(len(tickers_list1))+' tickers aftrer filtering')
toc=time.time()
      
print(str(toc-tic)+ ' s')


3840 tickers
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3499 tickers aftrer filtering
180.39995574951172 s


In [13]:
today=date.today()
s_today=str(today.year)+'-'+ str(today.month) +'-'+str(today.day)

start_date1 = today + relativedelta(weeks=-1)   #load last week of data
s_start_date1=str(start_date1.year)+'-'+ str(start_date1.month) +'-'+str(start_date1.day)

In [14]:
# pull all necessary data from SHARADAR/DAILY in one API call 

tic = time.time()


sharadar_daily_all= quandl.get_table('SHARADAR/DAILY',
                                      date={'gte':s_start_date1, 'lte': s_today},
                                      qopts={"columns":["ticker","ev", "marketcap"]},
                                      ticker=tickers_list1,
                                      paginate=True)

toc=time.time()
      
print(str(toc-tic)+ ' s')


2.276463270187378 s


In [15]:
# pull all necessary data from SHARADAR/SF1 in one API call

tic = time.time()


sharadar_sf1_all = quandl.get_table('SHARADAR/SF1',
                                ticker=tickers_list1,
                                qopts={"columns":["ticker","dimension","calendardate","fcf","revenueusd","marketcap"]},
                                paginate=True)


toc=time.time()
      
print(str(toc-tic)+ ' s')


72.66361165046692 s


In [16]:
# toss tickers that are not updated quarterly

tic=time.time()


print(str(len(tickers_list1))+' tickers')


i=0

for ticker in tickers_list1:
    i+=1
    
    s1=sharadar_sf1_all[sharadar_sf1_all['ticker'] == ticker][sharadar_sf1_all['dimension']=='MRQ'].shape[0]
       
    if s1 == 0:
        tickers_list1.remove(ticker)
        
        
    if i % 100 == 0:
        print(i)


print(str(len(tickers_list1))+' tickers after filtering')

toc=time.time()
      
print(str(toc-tic)+ ' s')





3499 tickers


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3092 tickers after filtering
130.43283247947693 s


In [17]:
#  fundamental data required is ev & marketcap from daily metrics
# https://www.quandl.com/tables/SFA/SHARADAR-DAILY
#  also needed is last 4 quarters (summed) of fcf & revenueusd
# from https://www.quandl.com/tables/SFA/SHARADAR-SF1
#  also need prior 4 quarters (summed) of revenueusd to calculate the revenue growth since 12 months ago
# this data is then used to create a custom_metric = fcf / ev + rev_growth
# the tickers are then sorted by this metric in decending order and percentile is calculated from 1 to 0


# eV, marketcap - most recent value



tic = time.time()


# create data frame

df=pd.DataFrame(columns = ['ticker', 'marketcap','revenueusd','revenueusd_growth',
                           'fcf','ev','marginable','custom_metric'])




print(str(len(tickers_list1))+' tickers')



i=0

for ticker in tickers_list1:
        
    i+=1
    
    sharadar_daily = sharadar_daily_all[sharadar_daily_all['ticker'] == ticker].reset_index()
    
    try:        
        ev=sharadar_daily['ev'][0]
    except IndexError:
        ev = float('NaN')
    
    try:
        marketcap=sharadar_daily['marketcap'][0]
    except IndexError:
        marketcap = float('NaN')
    
    
    sharadar_sf1 = sharadar_sf1_all[sharadar_sf1_all['ticker'] == ticker].reset_index()
    
    
    
    try:                # in case fcf not listed for ticker
        fcf = sharadar_sf1[sharadar_sf1['dimension']=='MRT']['fcf'].values.tolist()[0]
    except IndexError:
        fcf = float('NaN')
    
    revenueusd_list=sharadar_sf1[sharadar_sf1['dimension']=='MRT']['revenueusd'].values.tolist()
    
    try:
        revenueusd = revenueusd_list[0]
    except IndexError:
        revenueusd = float('NaN')
    
    try:      # in case there is no prior value for ticker
        revenueusd_prior = revenueusd_list[4]
    except IndexError:
        revenueusd_prior = 0
        
        
        
        
    marketcap1 = marketcap > 500
    marketcap2 = marketcap > 5000
    healthcare = (sharadar_tickers[sharadar_tickers['ticker'] == ticker]['sector'].values[0] == 'Healthcare')\
                  and (sharadar_tickers[sharadar_tickers['ticker'] == ticker]['industry'].values[0]!= 'Medical Care Facilities')
    marginable1 = marketcap1 and (not healthcare)
    marginable2 = marketcap2 and healthcare
    
    marginable = marginable1 or marginable2
       
    
        
    revenueusd_growth = revenueusd - revenueusd_prior
    
    try:
        custom_metric = fcf / (ev + revenueusd_growth ) 
    except TypeError:
        custom_metric = float('NaN')
        
   
    
    df = df.append({'ticker': ticker,
                    'marketcap':marketcap,
                    'revenueusd':revenueusd,
                    'revenueusd_growth':revenueusd_growth,
                    'fcf':fcf,
                    'ev':ev,
                    'marginable':marginable,
                    'custom_metric': custom_metric}, 
                     ignore_index=True)
    
    
    
      
    if i%100==0:
        print(i)
        
        
toc=time.time()

print(str(toc-tic)+' s')
    
    

    


3092 tickers
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
102.99932074546814 s


In [18]:
# toss if custom metric is NaN

print(df.shape[0],' before')

df = df[df['custom_metric'].notna()]

print(df.shape[0],' after')

3092  before
2889  after


In [19]:
# sort by custom metric

df = df.sort_values(by=['custom_metric'],ascending=False).reset_index(drop=True)


In [20]:
# calculate percentile

df['percentile'] = (1-df.index/df.shape[0])*100

df.head()



,ticker,marketcap,revenueusd,revenueusd_growth,fcf,ev,marginable,custom_metric,percentile
0,SYN,5.8,0.0,0.0,-13653000.0,-1.7,False,8.031176e+06,100.000000
1,BPTH,12.1,0.0,0.0,-10153000.0,-1.9,False,5.343684e+06,99.965386
2,ACET,55.8,0.0,0.0,-67903000.0,-15.3,False,4.438105e+06,99.930772
3,NTEC,8.2,0.0,0.0,-18908000.0,-5.1,False,3.707451e+06,99.896158
4,MLND,27.4,0.0,0.0,-38224000.0,-20.1,False,1.901692e+06,99.861544


In [23]:
# then for the stocks in the percentile > x, calculate the stock price total return over y days and z days

x = 60


df1=pd.merge(df, df_total_return, on='ticker')

df1=df1[df1['percentile'] > x]

# total return = (latest_close - n_day_close + sum of dividends since n) / n_day_close

df1['y_tot_return'] = (df1['latest_close']-df1['y_day_close']+df1['y_sum_dividends'])/df1['y_day_close'] 
df1['z_tot_return'] = (df1['latest_close']-df1['z_day_close']+df1['z_sum_dividends'])/df1['z_day_close'] 





In [24]:
df1.head()

,ticker,marketcap,revenueusd,revenueusd_growth,fcf,ev,marginable,custom_metric,percentile,latest_close,y_day_close,z_day_close,y_sum_dividends,z_sum_dividends,ATR,y_tot_return,z_tot_return
0,SYN,5.8,0.0,0.0,-13653000.0,-1.7,False,8.031176e+06,100.000000,0.299,0.333,0.317,0.0,0.0,0.037714,-0.102102,-0.056782
1,BPTH,12.1,0.0,0.0,-10153000.0,-1.9,False,5.343684e+06,99.965386,3.280,4.110,4.140,0.0,0.0,0.484857,-0.201946,-0.207729
2,ACET,55.8,0.0,0.0,-67903000.0,-15.3,False,4.438105e+06,99.930772,10.720,13.520,11.890,0.0,0.0,1.072929,-0.207101,-0.098402
3,NTEC,8.2,0.0,0.0,-18908000.0,-5.1,False,3.707451e+06,99.896158,2.290,5.020,5.180,0.0,0.0,0.493286,-0.543825,-0.557915
4,MLND,27.4,0.0,0.0,-38224000.0,-20.1,False,1.901692e+06,99.861544,1.440,1.800,1.600,0.0,0.0,0.143143,-0.200000,-0.100000


In [25]:
# save to file

df1.to_excel('output.xlsx')



In [29]:
# example: getting a value out of data frame

df1[df1['ticker'] == 'SYN']['marginable'].values[0]

False